# DSC 80: Lab 06

### Due Date: Tuesday May 11th, 11:59 PM

## Instructions
Much like in DSC 10, this Jupyter Notebook contains the statements of the problems and provides code and markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding work will be developed in an accompanying `lab*.py` file, that will be imported into the current notebook.

Labs and programming assignments will be graded in (at most) two ways:
1. The functions and classes in the accompanying python file will be tested (a la DSC 20),
2. The notebook will be graded (for graphs and free response questions).

**Do not change the function names in the `*.py` file**
- The functions in the `*.py` file are how your assignment is graded, and they are graded by their name. The dictionary at the end of the file (`GRADED FUNCTIONS`) contains the "grading list". The final function in the file allows your doctests to check that all the necessary functions exist.
- If you changed something you weren't supposed to, just use git to revert!

**Tips for working in the Notebook**:
- The notebooks serve to present you the questions and give you a place to present your results for later review.
- The notebook on *lab assignments* are not graded (only the `.py` file).
- Notebooks for PAs will serve as a final report for the assignment, and contain conclusions and answers to open ended questions that are graded.
- The notebook serves as a nice environment for 'pre-development' and experimentation before designing your function in your `.py` file.

**Tips for developing in the .py file**:
- Do not change the function names in the starter code; grading is done using these function names.
- Do not change the docstrings in the functions. These are there to tell you if your work is on the right track!
- You are encouraged to write your own additional functions to solve the lab! 
    - Developing in python usually consists of larger files, with many short functions.
    - You may write your other functions in an additional `.py` file that you import in `lab**.py` (much like we do in the notebook).
- Always document your code!

### Importing code from `lab**.py`

* We import our `.py` file that's contained in the same directory as this notebook.
* We use the `autoreload` notebook extension to make changes to our `lab**.py` file immediately available in our notebook. Without this extension, we would need to restart the notebook kernel to see any changes to `lab**.py` in the notebook.
    - `autoreload` is necessary because, upon import, `lab**.py` is compiled to bytecode (in the directory `__pycache__`). Subsequent imports of `lab**` merely import the existing compiled python.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import lab06 as lab

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import time

In [4]:
import requests
import bs4
import json

# Basic HTML tags practice

**Question 1**

Create a very basic `html` file that satisfies the following properties:

1. Has `<head>` and `<body>` tags.
2. Has a title
3. Inside the body tags:
    * At least two headers
    * At least three images:
        * At least one image must be a local file;
        * At least one image must be linked to online source; 
        * At least one image has to have default text when it cannot be displayed.
    * At least three references (hyperlinks) to different web pages;
    * At least one table with two columns.
    
        
   
4. Save your work as `lab06_1.html` in the same directory as `lab06.py`, make sure it loads in the browser and do not forget to submit it.
5. **Do not forget to submit all data files needed to display your page.**

**Note:** You can toy with (basic) HTML in the cells of a notebook, using either a "markdown cell" or by using the `IPython.display.HTML` function. However, be sure to open your saved file in a browser to be sure the page displays properly!

**Note:** If you work within Jupyter Notebook, you can later copy your text into a text editor and save it with the .html extension.

# Scraping an Online Bookstore


**Question 2**

Browse through the following fake on-line bookstore: http://books.toscrape.com/. This website is meant for toying with scraping.

Scrape the website, collecting data on all books that have **at least a four-star rating**, with a price **under £50** and belong to the book categories you want. You should collect the data in a dataframe as below (if you get an encoding error on your prices columns, like you see in the table below, don't worry about it):
<img src="data/bookdata.png">


Do this using the following steps:
1. Create a function `extract_book_links` that takes in the content of a book-listing page (a string of html), and returns a list of urls of book-detail pages that satisfy the requirements on "*at least* a four-star rating, and prices are *under* £50". 

2. Create a function `get_product_info` that takes in the content of a book-detail page (a string of html), a variable `categories` that is a list of book categories you want. If this input book is in the categories you want, returns a dictionary corresponding to a row in the dataframe in the image above (where the keys are the column names and the values are the row values); else, skip this book since this is not the book you want (ie. return None).

3. Create a function `scrape_books` of a single variable `k` that scrapes the first `k` pages of the bookstore (as determined by starting at the url above and clicking on the 'next' button),a variable `categories` that is a list of book categories you want, and returns a dataframe of books as the picture above. (Note: make sure the books returned satisfy the requirements set in part 1 about rating and price).


*Note:* Your function should take under 180 seconds to run through the entire bookstore.

*Note:* Don't worry about type casting (ie changing number of reviews to an int)

In [21]:
fp = os.path.join('data', 'products.html')
text = open(fp, encoding='utf-8').read()

In [22]:
bs = bs4.BeautifulSoup(text)

In [23]:
lab.extract_book_links(text)

['seven-brief-lessons-on-physics_219/index.html',
 'scarlet-the-lunar-chronicles-2_218/index.html',
 'saga-volume-3-saga-collected-editions-3_216/index.html',
 'running-with-scissors_215/index.html',
 'rise-of-the-rocket-girls-the-women-who-propelled-us-from-missiles-to-the-moon-to-mars_213/index.html',
 'ready-player-one_209/index.html']

In [24]:
fp = os.path.join('data', 'Frankenstein.html')
text = open(fp, encoding='utf-8').read()

In [25]:
q2_2 = lab.get_product_info(text,['Default'])

In [156]:
%%timeit
lab.scrape_books(49, ['Romance', 'Mystery'])

43.4 s ± 376 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# API Requests
**Question 3**

You trade stocks as a hobby. As an avid pandas coder, you figured it is best to calculate some statistics by pulling data from a public API (https://financialmodelingprep.com/developer/docs/#Stock-Historical-Price). Specifically, "Historical price with change and volume interval".

Some definitions (these are the ones you need to know):
- open: The opening price of a stock at the beginning of a trading day
- close: The closing price of a stock at the end of a trading day
- volume: The total number of shares being traded in a day
- percent change: difference in price with respect to the original price (in percentages)


1. Create a function `stock_history` which takes in the stock code (`ticker`) as a string, `year` and `month` as integers, and return a dataframe which has the price history for that stock in that month (include all columns).

2. Create a function `stock_stats` that takes in the output dataframe from `stock_history` and output the stock price change as a percentage and a rough total transaction volume **in billion dollars** for that month. Assume that on average, shares are traded at the midpoint price of high and low for that day. Return these two values as a tuple in a readable format: reserve 2 decimal points for both values and add a plus or minus sign at the front of the percent change. 
$$ \text{Total Transaction Volume (in dollars)} = \text{Volume (number of shares traded)} \times \text{Price} $$

*Example*: If \\$BYND opens at \\$80 and closes at \\$120 with a volume of 1 million, its percent change for the day is $(\$120-\$80) \div \$80 = +50.00\%$. And the estimated total transaction volume is: $(\$80+\$120) / 2 \times 10^6 = 0.10\text{B}$.


Hint: [pd.date_range](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.date_range.html), 

*Note:* Make sure you read the API documentation if you get stuck!

*Note 2:* In order to make successful requests, you will need an API key. In order to get one, you will need to sign up to the website. Once signed up, you can use the API key that comes with the free plan. It has a limit of 250 requests per day, which should be more than enough. In the code below, replace `your_key` when making requests.

In [392]:
q3_1 = lab.stock_history('AAPL', 2019, 6)

In [393]:
q3_2 = lab.stock_stats(q3_1)
q3_2

('+12.71%', '99.05B')

# Comment Threads

**Question 4**

As a hacker, you get your daily dose of tech news on [Hacker News](https://news.ycombinator.com/). The problem now is that you don't have internet access on your phone in your morning commute to work, so you want to save the interesting stories' comments thread beforehand in a flat file source like csv. You find their API documentation ( https://github.com/HackerNews/API) and implement the following task:

1. Write a function `get_comments` that takes `storyid` as a parameter and returns a dataframe of all the comments below the news story. You can ignore 'dead' comments (you will know it when you see it). **Make sure the order of the comments in your dataframe is from top to bottom just as you see on the website**. You are allowed to use loops in this function. Addtional requirement: write at least one helper method

You only want these information for the comments:
1. `id`: the unique ids
2. `by`: the author of the comment
3. `parent`: who (also in unique ids) they are replying to
4. `text`: the actual comment
5. `time`: when the comment is created (in `pd.datetime` format)

Hints:
1. Use depth-first-search when traversing the comments tree.
2. https://docs.python.org/3/tutorial/datastructures.html#using-lists-as-stacks.
3. Check the size of your dataframe to the story's `descendants` attribute (number of comments).

In [109]:
storyid = 18344932

In [110]:
news_endpoint = f"https://hacker-news.firebaseio.com/v0/item/{storyid}.json"

In [111]:
response = requests.get(news_endpoint).json()

In [112]:
response

{'by': 'ScottWRobinson',
 'descendants': 18,
 'id': 18344932,
 'kids': [18380397,
  18346406,
  18348601,
  18346750,
  18346476,
  18346746,
  18346388],
 'score': 47,
 'time': 1540987334,
 'title': 'TimescaleDB 1.0 Is Production Ready',
 'type': 'story',
 'url': 'https://blog.timescale.com/1-0-enterprise-production-ready-time-series-database-open-source-d32395a10cbf'}

In [121]:
graph = response
from collections import deque

In [108]:
next_id = response['kids']
ep = f"https://hacker-news.firebaseio.com/v0/item/{next_id}.json"
res = requests.get(ep).json()
res

{'error': 'Invalid path: Invalid token in path'}

In [122]:
data = []
def dfs(graph, data=None, q = None):
    if q is None:
        q = deque()
    if 'dead' in graph:
        q.append(graph)
    if 'kids' in graph:
        users = graph['kids']     
        q.append(graph)
        for user_id in users:
            end_point = f"https://hacker-news.firebaseio.com/v0/item/{user_id}.json"
            new_graph = requests.get(end_point).json()
            dfs(new_graph, data, q)
            q.append(new_graph)
    if 'dead' not in graph:
        data.append(q.popleft())

In [123]:
dfs(graph, data)

In [151]:
df = pd.DataFrame(data).iloc[1:].reset_index().filter(items = ['id', 'by', 'parent', 'text', 'time'])
df

,id,by,parent,text,time
0,18380397,valyala,18344932.0,TimescaleDB is great for storing time series c...,1541400799
1,18346406,msiggy,18344932.0,I&#x27;m excited to give this database a try i...,1540999222
2,18348601,sman393,18344932.0,Can this be used side by side on normal Postgr...,1541014179
3,18348631,RobAtticus,18348601.0,"Yep, absolutely. Regular PostgreSQL tables coe...",1541014492
4,18348984,sman393,18348631.0,Good to hear! how does the current TimescaleDB...,1541017426
5,18349540,RobAtticus,18348984.0,Not sure I follow exactly what you&#x27;re ask...,1541022440
6,18350673,sman393,18349540.0,Alright thanks! I thought I read that Timescal...,1541034719
7,18351061,RobAtticus,18350673.0,It does not support sharding writes across mul...,1541039703
8,18350673,sman393,18349540.0,Alright thanks! I thought I read that Timescal...,1541034719
9,18349540,RobAtticus,18348984.0,Not sure I follow exactly what you&#x27;re ask...,1541022440


In [145]:
df['time'] = df['time'].apply(lambda x: pd.to_datetime(x, unit='s'))

In [153]:
lab.get_comments(8863)

,id,by,parent,text,time
0,9224,BrandonM,8863.0,I have a few qualms with this app:<p>1. For a ...,2007-04-05 15:16:54
1,9272,dhouston,9224.0,"1. re: the first part, many people want someth...",2007-04-05 16:47:01
2,9479,BrandonM,9272.0,You are correct that this presents a very good...,2007-04-06 01:39:04
3,9272,dhouston,9224.0,"1. re: the first part, many people want someth...",2007-04-05 16:47:01
4,9224,BrandonM,8863.0,I have a few qualms with this app:<p>1. For a ...,2007-04-05 15:16:54
...,...,...,...,...,...
66,9353,dhouston,9324.0,couple of clarifications :)<p>1) i have other ...,2007-04-05 19:42:55
67,9324,vlad,9205.0,I totally missed that. I just know he was int...,2007-04-05 18:42:08
68,9205,zkinion,9097.0,"I'm guessing he did, though I may be wrong. T...",2007-04-05 14:54:19
69,9097,vlad,9067.0,I don't know if he applied; he has tried befor...,2007-04-05 06:41:28


## Congratulations! You're done!

* Submit the lab on Gradescope